# TVA sur les biens alimentaires

In [1]:
from __future__ import division

import os
import pandas as pd
import pkg_resources
import seaborn

from openfisca_france_indirect_taxation.surveys import SurveyScenario
from openfisca_france_indirect_taxation.build_survey_data.calibration_aliss import get_adjusted_input_data_frame


In [2]:
year = 2014
data_year = 2011
reform_key = 'aliss_tva_sociale'
scenario_kwargs = dict(year = year, data_year = 2011, reform_key = reform_key)
survey_scenario = SurveyScenario.create(**scenario_kwargs)
# 

In [3]:
adjusted_scenario_kwargs = dict(scenario_kwargs)
adjusted_scenario_kwargs.update(dict(
    data_year = None,
    input_data_frame = get_adjusted_input_data_frame(reform_key = reform_key[6:])
    ))
adjusted_survey_scenario = SurveyScenario.create(**adjusted_scenario_kwargs) 

kantar_expenditures_total:  52.035641344
bdf_expenditures_total:  52.218515456
population_kantar_total:  11968.2298584
bdf_expenditures_total: 

/home/benjello/openfisca/openfisca-france-indirect-taxation/openfisca_france_indirect_taxation/build_survey_data/calibration_aliss.py:119: FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  input_data_frame.eval("age = 0 + (agepr > 30) + (agepr > 45) + (agepr > 60)",
/home/benjello/openfisca/openfisca-survey-manager/openfisca_survey_manager/statshelpers.py:122: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  ret[and_(data >= lower, data < upper)] = labels[i]
/home/benjello/openfisca/openfisca-france-indirect-taxation/openfisca_france_indirect_taxation/build_survey_data/calibration_aliss.py:502: FutureWarning: eval expressions containing an assignment currentlydefault to operating inplace.
This will change in a future version of pandas, use inplace=True to avoid this warning.
  'adjusted_bdf_budget_share = kan

 126.629058796


In [4]:
alimentation_domicile_hors_alcool = [
    "depenses_ht_{}".format(key) for key in survey_scenario.tax_benefit_system.column_by_name.keys() \
    if key.startswith(u'poste_01')
    ]
alimentation_domicile = alimentation_domicile_hors_alcool + [
    'depenses_biere',
    'depenses_vin',
    'depenses_alcools_forts'
    ]

In [5]:
variables = alimentation_domicile + [
    'poste_agrege_01',
    'poste_agrege_02', 
    'depenses_ht_tva_taux_super_reduit',
    'depenses_ht_tva_taux_reduit',
    'depenses_ht_tva_taux_intermediaire',
    'depenses_ht_tva_taux_plein',
    'tva_taux_super_reduit',
    'tva_taux_reduit',
    'tva_taux_intermediaire',
    'tva_taux_plein',
    'tva_total',
    ]

In [6]:
pivot_table = pd.DataFrame()
reference_pivot_table = pd.DataFrame()
adjusted_pivot_table = pd.DataFrame()
aggregates = dict()
reference_aggregates = dict()
adjusted_aggregates = dict()

for variable in variables:
    pivot_table = pd.concat([
        pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    reference_pivot_table = pd.concat([
        reference_pivot_table,
        survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'],
                                           reference = True)
        ])
    adjusted_pivot_table = pd.concat([
        adjusted_pivot_table,
        adjusted_survey_scenario.compute_pivot_table(values = [variable], columns = ['niveau_vie_decile'])
        ])
    aggregates[variable] = survey_scenario.compute_aggregate(variable) / 1e9
    reference_aggregates[variable] = survey_scenario.compute_aggregate(variable, reference = True) / 1e9
    adjusted_aggregates[variable] = adjusted_survey_scenario.compute_aggregate(variable) / 1e9

In [7]:
assert sum(aggregates[aliment] - reference_aggregates[aliment] for aliment in alimentation_domicile) == 0

In [8]:
assert aggregates['poste_agrege_02'] - reference_aggregates['poste_agrege_02'] < 1e-7

In [9]:
for key in sorted(aggregates.keys()):
    if round(aggregates[key] - reference_aggregates[key], 3) != 0:
         print key, ': ', round(aggregates[key], 3), ' - ', round(reference_aggregates[key], 3), ' = ', round(aggregates[key] - reference_aggregates[key], 3)

depenses_ht_tva_taux_intermediaire :  183.132  -  75.594  =  107.538
depenses_ht_tva_taux_plein :  311.769  -  315.773  =  -4.004
depenses_ht_tva_taux_reduit :  24.994  -  128.528  =  -103.534
poste_agrege_01 :  128.399  -  124.14  =  4.259
tva_taux_intermediaire :  18.313  -  7.559  =  10.754
tva_taux_plein :  62.354  -  63.155  =  -0.801
tva_taux_reduit :  1.375  -  7.069  =  -5.694
tva_total :  82.183  -  77.924  =  4.259


In [10]:
for key in sorted(aggregates.keys()):
    if round(adjusted_aggregates[key] - aggregates[key], 3) != 0:
         print key, ': ', round(adjusted_aggregates[key], 3), ' - ', round(aggregates[key], 3), ' = ', round(adjusted_aggregates[key] - aggregates[key], 3)

depenses_alcools_forts :  2.923  -  2.932  =  -0.009
depenses_biere :  1.164  -  1.301  =  -0.138
depenses_ht_poste_01_10_1 :  2.868  -  2.924  =  -0.056
depenses_ht_poste_01_10_2 :  1.401  -  1.427  =  -0.025
depenses_ht_poste_01_1_1_1_1 :  13.391  -  13.376  =  0.015
depenses_ht_poste_01_1_1_3_3 :  3.757  -  3.73  =  0.028
depenses_ht_poste_01_1_1_4_1 :  1.478  -  1.488  =  -0.009
depenses_ht_poste_01_1_1_4_2 :  1.619  -  1.615  =  0.004
depenses_ht_poste_01_1_1_4_3 :  0.771  -  0.851  =  -0.08
depenses_ht_poste_01_1_2_1_1 :  7.43  -  7.457  =  -0.027
depenses_ht_poste_01_1_2_3_1 :  1.074  -  1.075  =  -0.001
depenses_ht_poste_01_1_2_4_1 :  2.166  -  2.176  =  -0.01
depenses_ht_poste_01_1_2_4_2 :  8.404  -  8.374  =  0.03
depenses_ht_poste_01_1_2_5_1 :  4.705  -  4.766  =  -0.061
depenses_ht_poste_01_1_2_6_1 :  0.49  -  0.471  =  0.019
depenses_ht_poste_01_1_2_6_3 :  2.971  -  2.943  =  0.028
depenses_ht_poste_01_1_3_1_1 :  2.762  -  2.841  =  -0.079
depenses_ht_poste_01_1_3_1_2 :  1

In [11]:
from openfisca_france_indirect_taxation.model.consommation.postes_coicop import get_legislation_data_frames
categories_fiscales_data_frame, codes_coicop_data_frame = get_legislation_data_frames()

In [12]:
aggregates['poste_agrege_01'] - reference_aggregates['poste_agrege_01']

4.2585982890594494

In [15]:
adjusted_pivot_table - pivot_table

niveau_vie_decile,1,2,3,4,5,6,7,8,9,10
depenses_ht_poste_01_1_1_1_1,10.076460,11.645364,12.008021,18.672933,10.858191,17.924659,6.039513,17.909620,9.959046,14.413260
depenses_ht_poste_01_1_1_3_3,8.805001,-0.224120,3.714574,7.270715,5.453082,3.346972,6.108335,0.831117,4.790954,4.523653
depenses_ht_poste_01_1_1_4_1,2.531350,-2.030909,2.302818,0.543523,3.332434,-0.723578,2.780144,-0.315760,1.303388,0.692565
depenses_ht_poste_01_1_1_4_2,2.309318,0.427252,1.670977,3.427391,1.845575,0.499832,1.945171,1.242746,1.894606,1.017368
depenses_ht_poste_01_1_1_4_3,-13.962576,-4.085584,-1.571524,0.312634,-0.228299,-0.958297,-0.511416,-0.022699,-0.059466,0.110475
depenses_ht_poste_01_1_2_1_1,1.536312,8.347624,4.635788,8.995629,10.225158,4.874309,8.461244,1.109264,3.862269,7.242193
depenses_ht_poste_01_1_2_3_1,4.499868,-2.247095,-0.989299,2.500762,1.374606,-2.264357,1.160431,3.090621,0.694427,1.834200
depenses_ht_poste_01_1_2_4_1,-3.403084,1.332132,1.068695,4.242586,1.281840,2.134223,4.144280,2.424931,1.721718,1.588485
depenses_ht_poste_01_1_2_4_2,10.436676,8.600416,2.371281,16.496801,9.028112,10.517251,2.105851,11.156408,8.167830,9.483255
depenses_ht_poste_01_1_2_5_1,-1.948868,-7.280804,2.259023,2.804330,6.300693,4.503185,1.015567,6.290797,2.565431,5.853960
